In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math


In [2]:
file = 'adult.csv'
df = pd.read_csv(file)
df.head(5)

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [3]:
names = list(df.columns)
print(names)

['39', ' State-gov', ' 77516', ' Bachelors', ' 13', ' Never-married', ' Adm-clerical', ' Not-in-family', ' White', ' Male', ' 2174', ' 0', ' 40', ' United-States', ' <=50K']


In [4]:
types = df.dtypes
print(types)

39                 int64
 State-gov        object
 77516             int64
 Bachelors        object
 13                int64
 Never-married    object
 Adm-clerical     object
 Not-in-family    object
 White            object
 Male             object
 2174              int64
 0                 int64
 40                int64
 United-States    object
 <=50K            object
dtype: object


In [5]:
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()
a = ['lan','lan','hung','diep']
le.fit_transform(a)
b = le.transform(a)
print(b)

[2 2 1 0]


In [6]:
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()
for i in range(len(types)):
  if types[i]=='object':
    le.fit_transform(df[names[i]])
    df[names[i]] = le.transform(df[names[i]])

In [7]:
data = df.values
X = data[:,:-1]
y = data[:,-1]
print(X.shape)
print(y.shape)

(32560, 14)
(32560,)


In [8]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
print(X_scaled[0])

[0.45205479 0.75       0.0482376  0.6        0.8        0.33333333
 0.28571429 0.         1.         1.         0.         0.
 0.12244898 0.95121951]


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(26048, 14)
(6512, 14)
(26048,)
(6512,)


Apply backpropagation algorithm with for training a MLP for classification using Cross Entropy Cost function 

In [10]:
# Apply backpropagation algorithms for training a MLP for classification using Cross-Entropy Cost function

def sigmoid(x):
  return 1/(1+np.exp(-x))

def sigmoid_derivative(x):
    return x*(1-x)

class NeuralNetwork_Classification:
    def __init__(self, layers, alpha=0.1):
		
      self.layers = layers 
      
      # Learning rate
      self.alpha = alpha
		
      # W, b
      self.W = []
      self.b = []
      
      # Wegihts and Bias in each layer
      for i in range(0, len(layers)-1):
            w_ = np.random.randn(layers[i], layers[i+1]) # weight
            b_ = np.zeros((layers[i+1], 1)) # bias
            self.W.append(w_/layers[i])
            self.b.append(b_)
            
    
	
	# Train model
    def fit_partial(self, x, y):
        A = [x]
        
        # Feedforward
        out = A[-1]
        for i in range(0, len(self.layers) - 1):
            out = sigmoid(np.dot(out, self.W[i]) + (self.b[i].T))
            A.append(out)
        
        # Backpropagation
        y = y.reshape(-1, 1)
        dA = [-(y/A[-1] - (1-y)/(1-A[-1]))]
        dW = []
        db = []
        for i in reversed(range(0, len(self.layers)-1)):
            dw_ = np.dot((A[i]).T, dA[-1] * sigmoid_derivative(A[i+1]))
            db_ = (np.sum(dA[-1] * sigmoid_derivative(A[i+1]), 0)).reshape(-1,1)
            dA_ = np.dot(dA[-1] * sigmoid_derivative(A[i+1]), self.W[i].T)
            dW.append(dw_)
            db.append(db_)
            dA.append(dA_)
        
        # Inverted dW, db
        dW = dW[::-1]
        db = db[::-1]
        
		# Gradient descent
        for i in range(0, len(self.layers)-1):
            self.W[i] = self.W[i] - self.alpha * dW[i]
            self.b[i] = self.b[i] - self.alpha * db[i]
      
    def fit(self, X, y, epochs=20, verbose=10):
        for epoch in range(0, epochs):
            self.fit_partial(X, y)
            if epoch % verbose == 0:
                loss = self.calculate_loss(y, self.predict(X))
                print("Epoch {}, loss {}".format(epoch, loss))
    
	# Prediction
    def predict(self, X):
        for i in range(0, len(self.layers) - 1):
            X = sigmoid(np.dot(X, self.W[i]) + (self.b[i].T))
        return X

	# Cross entropy loss function
    def calculate_loss(self, y, y_pre):
      loss=-np.sum(y*np.log(y_pre))
      return loss/float(y_pre.shape[0])
        


In [11]:
# Train model
nn = NeuralNetwork_Classification([X_train.shape[1], 32, 1], alpha=0.1)
nn.fit(X_train, y_train, epochs=100, verbose=10)
    

C:\Users\ASUS\AppData\Local\Temp\ipykernel_15004\1775882698.py:4: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
C:\Users\ASUS\AppData\Local\Temp\ipykernel_15004\1775882698.py:77: RuntimeWarning: divide by zero encountered in log
  loss=-np.sum(y*np.log(y_pre))
C:\Users\ASUS\AppData\Local\Temp\ipykernel_15004\1775882698.py:77: RuntimeWarning: invalid value encountered in multiply
  loss=-np.sum(y*np.log(y_pre))


Epoch 0, loss nan


C:\Users\ASUS\AppData\Local\Temp\ipykernel_15004\1775882698.py:42: RuntimeWarning: divide by zero encountered in true_divide
  dA = [-(y/A[-1] - (1-y)/(1-A[-1]))]
C:\Users\ASUS\AppData\Local\Temp\ipykernel_15004\1775882698.py:42: RuntimeWarning: invalid value encountered in true_divide
  dA = [-(y/A[-1] - (1-y)/(1-A[-1]))]
C:\Users\ASUS\AppData\Local\Temp\ipykernel_15004\1775882698.py:46: RuntimeWarning: invalid value encountered in multiply
  dw_ = np.dot((A[i]).T, dA[-1] * sigmoid_derivative(A[i+1]))
C:\Users\ASUS\AppData\Local\Temp\ipykernel_15004\1775882698.py:47: RuntimeWarning: invalid value encountered in multiply
  db_ = (np.sum(dA[-1] * sigmoid_derivative(A[i+1]), 0)).reshape(-1,1)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_15004\1775882698.py:48: RuntimeWarning: invalid value encountered in multiply
  dA_ = np.dot(dA[-1] * sigmoid_derivative(A[i+1]), self.W[i].T)


Epoch 10, loss nan
Epoch 20, loss nan
Epoch 30, loss nan
Epoch 40, loss nan
Epoch 50, loss nan
Epoch 60, loss nan
Epoch 70, loss nan
Epoch 80, loss nan
Epoch 90, loss nan


In [12]:
# Prediction
y_pre = nn.predict(X_test)
y_pre = y_pre.reshape(-1,1)
y_pre = (y_pre>0.5).astype(int)
print(y_pre.shape)
print(y_test.shape)
    

(6512, 1)
(6512,)


In [13]:
# Accuracy
from sklearn.metrics import accuracy_score
print("Accuracy score: ",accuracy_score(y_test, y_pre))

0.7542997542997543
